In [1]:
# Analysis of cytokine perturbation scores, only includes mesenchymal cells

In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np

import squidpy as sq
import matplotlib.pyplot as plt
from matplotlib import rcParams

In [ ]:
sc.settings.figdir = "./figures_cytokines/"

!mkdir figures_cytokines

In [ ]:
#adata = sc.read_h5ad("h5ad/scvi/xenium5k_scVI_CellCharter.h5ad")
adata = sc.read_h5ad("h5ad/scvi/xenium5k_scVI_CellCharter_clustImmune_postQC.h5ad")

In [ ]:
adata

In [ ]:
adata.obs.annot2.value_counts()

In [ ]:
#print(adata.X[1:10, 1:10])

In [ ]:
# Mesenchymal subset
adata = adata[adata.obs.megagut_lvl1_majority_voting == 'Mesenchymal']

In [ ]:
adata

In [ ]:
# Exclude cells that are not consistent with mesenchymal compartment according to level 3 CellTypist predictions
exclude_cells = ["Enterocyte", "Tnaive/cm_CD4", "B_memory", "Glial/Enteric_neural_crest", "Trm_CD8", "Treg_IL10", "EC_capillary", "EC_lymphatic", "Trm_CD4", "Mesoderm_2"]

adata = adata[~adata.obs.megagut_lvl3_majority_voting.isin(exclude_cells)]

In [ ]:
adata

In [ ]:
adata.obs.annot.value_counts()

In [ ]:
# Recompute UMAPs for mesenchymal cells

In [ ]:
sc.pp.neighbors(adata, use_rep="X_scVI")

In [ ]:
# UMAP
sc.tl.umap(adata)

# Load bulk RNA-seq signatures

In [ ]:
def load_de_genes(fn, sep=',', max_p=0.01, min_lfc=2, max_n=1000):
    de_df = pd.read_csv(fn, sep=sep, index_col=0)
    de_df = de_df.where(lambda x: (x.padj < max_p) & (x.log2FoldChange > np.log2(min_lfc))).dropna().sort_values(['padj', 'log2FoldChange'], ascending=[True, False])
    return de_df.iloc[0:max_n, :]

In [ ]:
!ls ../fibroblast_reanalysis_bulk_DE_signature/bulkRNA_DE_v2/csv/*.csv

In [ ]:
rna_stimulations = !ls ../fibroblast_reanalysis_bulk_DE_signature/bulkRNA_DE_v2/csv/ | cut -f2 -d '_'
rna_stimulations = [f.strip(".csv") for f in rna_stimulations]

In [ ]:
rna_stimulations

In [ ]:
rna_up_genes = {rstim: load_de_genes(f'../fibroblast_reanalysis_bulk_DE_signature/bulkRNA_DE_v2/csv/DESeq2_{rstim}.csv', sep=',', max_p=0.05, min_lfc=1.5) for rstim in rna_stimulations}

In [ ]:
# Count number of genes
{k: len(v) for k,v in rna_up_genes.items()}

In [ ]:
#adata.var.index

In [ ]:
# Score signatures
for rstim, rup in rna_up_genes.items():
    g = rup.index.to_list()  # gene set
    
    # Subset to overlap only
    g = set(g).intersection(adata.var.index)
    
    print(rstim, ": ", len(g))
    sc.tl.score_genes(adata, g, ctrl_size=len(g), score_name=f'{rstim}_upreg')

In [ ]:
for rstim in rna_up_genes.keys():
    x = adata.obs[f'{rstim}_upreg'].values
    adata.obs[f'{rstim}_upreg_scaled'] = (x-x.mean()) / np.std(x)

In [ ]:
adata

In [ ]:
# Plot standardized scores
rna_up_genes_show = rna_up_genes.copy()
del rna_up_genes_show['IL6']
#del rna_up_genes_show['IL22']

sc.set_figure_params(dpi_save=300)

#sk.set_figsize((3, 3))
sc.pl.embedding(
    adata,
    basis='X_umap',
    color=[f'{rstim}_upreg_scaled' for rstim in rna_up_genes_show.keys()] + ['annot'],
    ncols=5,
    #cmap='viridis',
    palette=sc.pl.palettes.default_102,
    cmap='RdBu_r',
    title=[f'{rstim}_upreg, n={rna_up_genes[rstim].shape[0]}' for rstim in rna_up_genes_show.keys()] + ['Cell type'],
    vmin=-4, vmax=4,
    #vcenter=0,
    save="_xenium5k_bulkDE_signature_projection_scaled_stromal.pdf"
)

In [ ]:
sc.pl.matrixplot(
    adata, [f'{rstim}_upreg_scaled' for rstim in rna_up_genes_show.keys()], groupby='annot', figsize=(7, 2.0), swap_axes=True,
    colorbar_title="Mean cytokine score",
    cmap='RdBu_r', vmin=-2, vmax=2,
    save="cytokines_cell_type_stromal.pdf"
)

In [ ]:
sc.pl.matrixplot(
    adata, [f'{rstim}_upreg_scaled' for rstim in rna_up_genes_show.keys()], groupby='spatial_cluster18', figsize=(5, 2.0), swap_axes=True,
    colorbar_title="Mean cytokine score",
    cmap='PRGn', vmin=-2, vmax=2,
    save="cytokines_CellCharter_niches_stromal.pdf"
)

In [ ]:
# rcParams["figure.figsize"] = (8, 4)
# sc.pl.violin(adata, [f'{rstim}_upreg_scaled' for rstim in rna_up_genes_show.keys()], groupby='spatial_cluster18', )

In [ ]:
adata

In [ ]:
# Formating
adata.obs.sample = adata.obs['sample']

In [ ]:
for column in [f'{rstim}_upreg_scaled' for rstim in rna_up_genes_show.keys()]:
    print(column)

In [ ]:
rcParams["figure.figsize"] = (6, 6)
sc.set_figure_params(dpi_save=450)

sq.pl.spatial_scatter(
    adata, 
    color='IFNA_upreg_scaled', 
    library_key='sample', 
    img=None, 
    title=adata.obs.sample.cat.categories,
    size=8,  # point size
    ncols=4,
    cmap='RdBu_r', vmin=-4, vmax=4, 
    library_id=adata.obs.sample.cat.categories,
#    save="figures_niches/CellCharter_niches18.pdf", dpi=450
)

In [ ]:
# rcParams["figure.figsize"] = (6, 6)
# sc.set_figure_params(dpi_save=450)

# sq.pl.spatial_scatter(
#     adata, 
#     color='IFNG_upreg_scaled', 
#     library_key='sample', 
#     img=None, 
#     title=adata.obs.sample.cat.categories,
#     size=8,  # point size
#     ncols=4,
#     cmap='RdBu_r', vmin=-4, vmax=4, 
#     library_id=adata.obs.sample.cat.categories,
# #    save="figures_niches/CellCharter_niches18.pdf", dpi=450
# )

In [ ]:
rcParams["figure.figsize"] = (6, 6)
sc.set_figure_params(dpi_save=450)

sq.pl.spatial_scatter(
    adata, 
    color='IL13_upreg_scaled', 
    library_key='sample', 
    img=None, 
    title=adata.obs.sample.cat.categories,
    size=8,  # point size
    ncols=4,
    cmap='RdBu_r', vmin=-4, vmax=4, 
    library_id=adata.obs.sample.cat.categories,
#    save="figures_niches/CellCharter_niches18.pdf", dpi=450
)

In [ ]:
rcParams["figure.figsize"] = (6, 6)
sc.set_figure_params(dpi_save=450)

sq.pl.spatial_scatter(
    adata, 
    color='OSM_upreg_scaled', 
    library_key='sample', 
    img=None, 
    title=adata.obs.sample.cat.categories,
    size=8,  # point size
    ncols=4,
    cmap='RdBu_r', vmin=-4, vmax=4, 
    library_id=adata.obs.sample.cat.categories,
#    save="figures_niches/CellCharter_niches18.pdf", dpi=450
)

In [ ]:
rcParams["figure.figsize"] = (6, 6)
sc.set_figure_params(dpi_save=450)

sq.pl.spatial_scatter(
    adata, 
    color='IL1B_upreg_scaled', 
    library_key='sample', 
    img=None, 
    title=adata.obs.sample.cat.categories,
    size=8,  # point size
    ncols=4,
    cmap='RdBu_r', vmin=-4, vmax=4, 
    library_id=adata.obs.sample.cat.categories,
#    save="figures_niches/CellCharter_niches18.pdf", dpi=450
)

In [ ]:
# Spatial plots of all cytokine signature scores
rcParams["figure.figsize"] = (6, 6)
sc.set_figure_params(dpi_save=300)

for column in [f'{rstim}_upreg_scaled' for rstim in rna_up_genes_show.keys()]:
    print(column)
    sq.pl.spatial_scatter(
        adata, 
        color=column, 
        library_key='sample', 
        img=None, 
        title=adata.obs.sample.cat.categories,
        size=8,  # point size
        ncols=4,
        cmap='RdBu_r', vmin=-4, vmax=4, 
        library_id=adata.obs.sample.cat.categories,
        save="spatial/spatial_cytokine_ " + column + "_stromal.pdf",
        dpi=300
    )

In [ ]:
#adata.obs.sample

In [ ]:
# Encode specific cell types as yes/no
adata.obs['S5'] = pd.NA
adata.obs.S5[adata.obs.annot == "S5"] = 'Yes'

In [ ]:
adata.obs['S3_oral_mucosa-like'] = pd.NA
adata.obs['S3_oral_mucosa-like'][adata.obs.annot == "S3_oral_mucosa-like"] = 'Yes'

In [ ]:
#del adata.uns['S5_colors']

In [ ]:
rcParams["figure.figsize"] = (6, 6)
sc.set_figure_params(dpi_save=450)

sq.pl.spatial_scatter(
    adata, 
    color=[
        'IL1B_upreg_scaled', 'IFNA_upreg_scaled', 'IL17A_upreg_scaled', 'OSM_upreg_scaled', 'IL13_upreg_scaled', 'TNF_upreg_scaled', 'TGFB1_upreg_scaled', 'IFNG_upreg_scaled',
        'spatial_cluster18', 'S5', 'S3_oral_mucosa-like', 'annot'],
    crop_coord=(2500, 5500, 4200, 10000),   # x0, y0, x1, y1
    library_key='sample', 
    img=None, 
#    title=adata.obs.sample.cat.categories,
    size=6,  # point size
    ncols=4,
    cmap='RdBu_r', vmin=-4, vmax=4, 
    library_id="0033516_affected",
    na_color='lightgrey',
    save="spatial/cytokines_view1_0033516_affected_stromal.pdf",
    dpi=450
)

In [ ]:
# rcParams["figure.figsize"] = (6, 6)

# sq.pl.spatial_scatter(
#     adata, 
#     color='annot',
#     groups=['S5', 'S3_oral_mucosa-like', 'EC_venous',
#             #'Pericyte_1',
#             #'EC_capillary',
#  #           'B_naive',
#  #           'Enterocyte'
#             'Tnaive/cm_CD4',
#             'B_plasma_IgA1',
#             'B_memory',
#             'Treg_IL10'
#            ],
#     crop_coord=(2500, 5500, 4200, 10000),   # x0, y0, x1, y1
#     library_key='sample', 
#     img=None, 
#     size=6,  # point sizea
#     ncols=4,
#     palette="tab10",
#     library_id="0033516_affected",
#     na_color='lightgrey',
#     save="spatial/cytokines_view1_0033516_affected_selectedCells.pdf",
#     dpi=450
# )

In [ ]:
# adata.obs.sample.value_counts()

In [ ]:
# adata

In [ ]:
# rcParams["figure.figsize"] = (6, 6)
# sc.set_figure_params(dpi_save=300)

# sq.pl.spatial_scatter(
#     adata, 
#     color=[
#         'IL1B_upreg_scaled', 'IFNA_upreg_scaled', 'IL17A_upreg_scaled', 'OSM_upreg_scaled', 'IL13_upreg_scaled', 'TNF_upreg_scaled', 'TGFB1_upreg_scaled',
#         'spatial_cluster18', 'S5', 'S3_oral_mucosa-like', 'annot'],
#     crop_coord=(4500, 3500, 8000, 5000),   # x0, y0, x1, y1
#     library_key='sample', 
#     img=None, 
# #    title=adata.obs.sample.cat.categories,
#     size=6,  # point size
#     ncols=4,
#     cmap='RdBu_r', vmin=-4, vmax=4, 
#     library_id="0043587_affected",
#     na_color='lightgrey',
#     save="spatial/cytokines_view1_0043587_affected.pdf",
#     dpi=300
# )

In [ ]:
# rcParams["figure.figsize"] = (6, 6)

# sq.pl.spatial_scatter(
#     adata, 
#     color='annot',
#     groups=['S5',
#             'S3_oral_mucosa-like',
#             'EC_venous',
#   #          'Pericyte_1',
# #             'EC_capillary',
# #             'B_naive',
# #  #           'Enterocyte'
#              'Tnaive/cm_CD4',
#              'B_plasma_IgA1',
#              'B_memory',
#              'Treg_IL10',
#  #           'Tfh', 'S4', 'DC_pDC'
#             'Trm_Th17', 'DC_migratory', 'DC_cDC2'
#            ],
#     crop_coord=(4500, 3500, 8000, 5000),   # x0, y0, x1, y1
#     library_key='sample', 
#     img=None, 
#     size=6,  # point size
#     ncols=4,
#     palette="tab10",
#     library_id="0043587_affected",
#     na_color='lightgrey',
#     save="spatial/cytokines_view1_0043587_affected_selectedCells.pdf",
#     dpi=450
# )